## Table and Graph Extraction

This notebook focuses on extraction of tables and graphs, in order to prove the idea, test articles will be used as example

In [6]:
import logging
import re
import pandas as pd
import urllib
import time
# import feedparser
import chemdataextractor as cde
from chemdataextractor import Document
import chemdataextractor.model as model
from chemdataextractor.model import Compound, UvvisSpectrum, UvvisPeak, BaseModel, StringType, ListType, ModelType
from chemdataextractor.parse.common import hyphen
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first
from chemdataextractor.parse.actions import strip_stop
from chemdataextractor.parse.elements import W, I, T, R, Optional, ZeroOrMore, OneOrMore
from chemdataextractor.parse.cem import chemical_name
from chemdataextractor.doc import Paragraph, Sentence, Caption, Figure,Table
from chemdataextractor.doc.table import Table, Cell

In [2]:
# open and read files
f = open('test_articles/paper0.pdf', 'rb')
doc = Document.from_file(f)
abstract = [11]

f1 = open('test_articles/paper1.pdf', 'rb')
doc1 = Document.from_file(f1)
abstract1 = [7,8]

f2 = open('test_articles/paper2.pdf', 'rb')
doc2 = Document.from_file(f2)
abstract2 = [7,8]

f3 = open('test_articles/paper3.pdf', 'rb')
doc3 = Document.from_file(f3)
abstract3 = [10]

f4 = open('test_articles/paper4.pdf', 'rb')
doc4 = Document.from_file(f4)
abstract4 = [12]

f5 = open('test_articles/paper5.pdf', 'rb')
doc5 = Document.from_file(f5)
abstract5 = [3,4]

f6 = open('test_articles/paper6.pdf', 'rb')
doc6 = Document.from_file(f6)
abstract6 = [5,6,7,8]

f7 = open('test_articles/paper7.pdf', 'rb')
doc7 = Document.from_file(f7)
abstract7 = [11]

In [3]:
# split the paragraph into elements
paras = doc.elements
cems = doc.cems
doc.records.serializealize()

[{'names': ['Isoindigo-']},
 {'names': ['bislactam']},
 {'names': ['hydrogens']},
 {'names': ['phenyl']},
 {'names': ['oxygens']},
 {'names': ['oxindoles']},
 {'names': ['triphenylamine']},
 {'names': ['phenyl- carbazole']},
 {'names': ['D − A']},
 {'names': ['4,8-bis(5-(2-ethylhexyl)- thiophen-2-yl)benzo[1,2-b:4,5-b′]dithiophene ( 2D-BDT )']},
 {'names': ['2D-BDT-containing D − π − A']},
 {'names': ['Mn [ kg mol−1 ]']},
 {'names': ['411']},
 {'names': ['34']},
 {'names': ['76']},
 {'names': ['long, branched 2- octyldodecyl alkyl']},
 {'names': ['tris(dibenzylideneacetone)dipalladium']},
 {'names': ['Mn']},
 {'names': ['benzene']},
 {'names': ['ferrocene']},
 {'names': ['−[Eonset ferrocene + 4.8 ] V']},
 {'names': ['methyl substituted alkyl chains']},
 {'names': ['methyl']},
 {'names': ['I ds']},
 {'names': ['WC o L 2']},
 {'names': ['V t']},
 {'names': ['PBDT-TIIG-']},
 {'names': ['P3HT']},
 {'names': ['PCBM']},
 {'names': ['μ e']},
 {'names': ['f [V]']},
 {'names': ['alkyl']},
 {'nam

### Built-in Test Examples in CDE

In [7]:
t = Table(
            caption=Caption('Selected photophysical properties of biarylsubstituted pyrazoles 5–8 and 1-methyl-3,5-diphenylpyrazole (9) at room temperature'),
            headings=[
                [
                    Cell('Compound'),
                    Cell('Absorption maxima λmax,abs (ε) [nm] (L cm−1 mol−1)'),
                    Cell('Emission maxima λmax,em (Φf) [nm] (a.u.)'),
                    Cell('Stokes-shift Δṽ [cm−1]')
                ]
            ],
            rows=[
                [Cell(' 5a '), Cell('273.5 (40 100)'), Cell('357.0 (0.77)'), Cell('9400')],
                [Cell(' 5b '), Cell('268.5 (36 700)'), Cell('359.0 (0.77)'), Cell('8600')],
                [Cell('Coumarin 343'), Cell('263.0 (38 400)'), Cell('344.5 (0.67)'), Cell('9000')],
                [Cell(' 5d '), Cell('281.0 (34 200)'), Cell('351.5 (0.97)'), Cell('7100')],
                [Cell(' 5e '), Cell('285.0 (44 000)'), Cell('382.0 (0.35)'), Cell('8900')],
                [Cell(' 5f '), Cell('289.0 (43 300)'), Cell('363.0 (0.80)'), Cell('7100')],
                [Cell(' 5g '), Cell('285.0 (42 000)'), Cell('343.5 (0.86)'), Cell('6000')],
                [Cell(' 6a '), Cell('283.5 (35 600)'), Cell('344.5 (0.49)'), Cell('6300')],
                [Cell(' 6b '), Cell('267.5 (35 800)'), Cell('338.5 (0.83)'), Cell('7800')],
                [Cell(' 6c '), Cell('286.0 (33 000)'), Cell('347.0 (0.27)'), Cell('6200')],
                [Cell(' 6d '), Cell('306.5 (36 600)'), Cell('384.0 (0.10)'), Cell('6600')],
                [Cell(' 7 '), Cell('288.5 (62 500)'), Cell('367.0 (0.07)'), Cell('7400')],
                [Cell('Compound 8a '), Cell('257.0 (36 300), 293.0 sh (25 000)'), Cell('385.0 (0.41)'), Cell('8200')],
                [Cell(' 8b '), Cell('257.0 (32 000), 296.0 sh (23000)'), Cell('388.0 (0.33)'), Cell('8000')],
                [Cell(' 8c '), Cell('257.0 (27 400), 307.5 (18900)'), Cell('387.0 (0.12)'), Cell('6700')],
                [Cell(' 8d '), Cell('268.5 (29 500)'), Cell('385.0 (0.29)'), Cell('11 300')],
                [Cell('Dye 8e '), Cell('261.5 (39 900), 288.0 sh (29 600), 311.0 sh (20 500)'), Cell('386.5 (0.37)'), Cell('6300')],
                [Cell(' 8f '), Cell('256.5 (27 260), 296.0 (28404)'), Cell('388.5 (0.35)'), Cell('8000')],
                [Cell(' 8g '), Cell('272.5 (39 600)'), Cell('394.0 (0.30)'), Cell('11 300')],
                [Cell(' 8h '), Cell('286.0 (22 900)'), Cell('382.5 (0.33)'), Cell('8800')],
                [Cell(' 9 '), Cell('254.0 (28 800)'), Cell('338.5 (0.40)'), Cell('9800')]]
)

In [8]:
t

Compound,"Absorption maxima λmax,abs (ε) [nm] (L cm−1 mol−1)","Emission maxima λmax,em (Φf) [nm] (a.u.)",Stokes-shift Δṽ [cm−1]
5a,273.5 (40 100),357.0 (0.77),9400
5b,268.5 (36 700),359.0 (0.77),8600
Coumarin 343,263.0 (38 400),344.5 (0.67),9000
5d,281.0 (34 200),351.5 (0.97),7100
5e,285.0 (44 000),382.0 (0.35),8900
5f,289.0 (43 300),363.0 (0.80),7100
5g,285.0 (42 000),343.5 (0.86),6000
6a,283.5 (35 600),344.5 (0.49),6300
6b,267.5 (35 800),338.5 (0.83),7800
6c,286.0 (33 000),347.0 (0.27),6200


In [14]:
[record.serialize() for record in t.records]

[{'labels': ['5a'],
  'uvvis_spectra': [{'peaks': [{'value': '273.5',
      'units': 'nm',
      'extinction': '40100',
      'extinction_units': 'L cm − 1 mol − 1'}]}],
  'quantum_yields': [{'value': '0.77'}]},
 {'labels': ['5b'],
  'uvvis_spectra': [{'peaks': [{'value': '268.5',
      'units': 'nm',
      'extinction': '36700',
      'extinction_units': 'L cm − 1 mol − 1'}]}],
  'quantum_yields': [{'value': '0.77'}]},
 {'names': ['Coumarin 343'],
  'uvvis_spectra': [{'peaks': [{'value': '263.0',
      'units': 'nm',
      'extinction': '38400',
      'extinction_units': 'L cm − 1 mol − 1'}]}],
  'quantum_yields': [{'value': '0.67'}]},
 {'labels': ['5d'],
  'uvvis_spectra': [{'peaks': [{'value': '281.0',
      'units': 'nm',
      'extinction': '34200',
      'extinction_units': 'L cm − 1 mol − 1'}]}],
  'quantum_yields': [{'value': '0.97'}]},
 {'labels': ['5e'],
  'uvvis_spectra': [{'peaks': [{'value': '285.0',
      'units': 'nm',
      'extinction': '44000',
      'extinction_units

In [15]:
# in paper0, the table is shown below
t2 = Table(
    caption = Caption("Table 1. Physicochemical Properties of the Study Polymers"),
    headings=[
                [
                    Cell('Polymer'),
                    Cell('Mn (kg/mol)'),
                    Cell('PDI'),
                    Cell('Tg (C)'),
                    Cell('Td (C)'),
                    Cell('soluton/λmax [nm]'),
                    Cell('film/λmax [nm]'),
                    Cell('HOMO [eV]'),
                    Cell('LUMO [eV]'),
                    Cell('Egec[eV]'),
                    Cell('Egopt[eV]'),
                ]
    ],
    
    rows=[
                [Cell(' PBDT-IIG '), Cell('21'), Cell('2.6'), Cell('56'),Cell('380'),Cell('359, 446, 625'),Cell('367, 456, 631, 678'),Cell('-5.38'),Cell('-5.35'),Cell('2.03'),Cell('1.59')],
                [Cell(' PBDT-TIIG '), Cell('34'), Cell('2.8'), Cell('76'),Cell('411'),Cell('463, 826,854'),Cell('466, 833,856'),Cell('-4.96'),Cell('-3.29'),Cell('1.67'),Cell('1.05')]
    ]

)

In [16]:
t2 # illustration from paper0.pdf

Polymer,Mn (kg/mol),PDI,Tg (C),Td (C),soluton/λmax [nm],film/λmax [nm],HOMO [eV],LUMO [eV],Egec[eV],Egopt[eV]
PBDT-IIG,21,2.6,56,380,"359, 446, 625","367, 456, 631, 678",-5.38,-5.35,2.03,1.59
PBDT-TIIG,34,2.8,76,411,"463, 826,854","466, 833,856",-4.96,-3.29,1.67,1.05


In [17]:
# output result to json format
[record.serialize() for record in t2.records]

[{'names': ['PBDT-IIG'],
  'uvvis_spectra': [{'peaks': [{'value': '359', 'units': 'nm'},
     {'value': '446', 'units': 'nm'},
     {'value': '625', 'units': 'nm'}]},
   {'peaks': [{'value': '367', 'units': 'nm'},
     {'value': '456', 'units': 'nm'},
     {'value': '631', 'units': 'nm'},
     {'value': '678', 'units': 'nm'}]}]},
 {'names': ['PBDT-TIIG'],
  'uvvis_spectra': [{'peaks': [{'value': '463', 'units': 'nm'}]},
   {'peaks': [{'value': '466', 'units': 'nm'}]}]}]

In [19]:
t3 = Table(caption = Caption("Table 3. Photovoltaic Parameters of Optimized Solar Cells"),
           headings = [
               [
                   Cell('polymer'),
                   Cell('polymer:PC71BM'),
                   Cell('Voc [V]'),
                   Cell('Jsc [mA cm-2]'),
                   Cell('FF [%]'),
                   Cell('PCE [%]'),
               ]
           ],
            
           rows = [
               [Cell(' PBDT-IIG '), Cell('21'), Cell('2.6'), Cell('56'),Cell('380'),Cell('359, 446, 625'),Cell('367, 456, 631, 678'),Cell('-5.38'),Cell('-5.35'),Cell('2.03'),Cell('1.59')],
               [Cell(' PBDT-TIIG '), Cell('21'), Cell('2.6'), Cell('56'),Cell('380'),Cell('359, 446, 625'),Cell('367, 456, 631, 678'),Cell('-5.38'),Cell('-5.35'),Cell('2.03'),Cell('1.59')],
           ]
    )

In [21]:
[record.serialize() for record in t3.records]

[{'names': ['PBDT-IIG']}, {'names': ['PBDT-TIIG']}]

Based on the test we can examine that the result is valid, all numbers are accurately extracted. Tabulated data is much easier to extract compare to graphs and texts.

In [18]:
paras

[Paragraph(id=None, references=[], text='Article'),
 Paragraph(id=None, references=[], text='pubs.acs.org/cm'),
 Paragraph(id=None, references=[], text='Interplay of Molecular Orientation, Film Formation, and\nOptoelectronic Properties on Isoindigo- and Thienoisoindigo-Based\nCopolymers for Organic Field Eﬀect Transistor and Organic\nPhotovoltaic Applications\nChien Lu,\nand Pi-Tai Chou*,#\n†\nDepartment of Chemical Engineering, National Taiwan University, Taipei 106, Taiwan\n‡\nResearch Center for New Generation Photovoltaics, Graduate Institute of Energy Engineering, National Central University, Taoyuan\n320, Taiwan\n#Department of Chemistry, National Taiwan University, Taipei 106, Taiwan'),
 Paragraph(id=None, references=[], text='Hsieh-Chih Chen,*,‡,§'),
 Paragraph(id=None, references=[], text='Wen-Chang Chen,*,†'),
 Paragraph(id=None, references=[], text='Wei-Ti Chuang,'),
 Paragraph(id=None, references=[], text='Yen-Hao Hsu,'),
 Paragraph(id=None, references=[], text='†,§'),
 Par